In [1]:
import overpy
import shapely.geometry
import geopandas as gpd

# 1. 定义上海边界（这里用一个简化的 bbox 例子；你可以用精确行政边界多边形）
min_lon, min_lat = 120.85, 30.65
max_lon, max_lat = 122.20, 31.90

# 2. 设置 Overpass API
api = overpy.Overpass()

# 3. 构造 Overpass QL 查询
# 这个例子获取在 bbox 内，标签为 amenity 的 POI
query = f"""
(
  node["amenity"]({min_lat},{min_lon},{max_lat},{max_lon});
  way["amenity"]({min_lat},{min_lon},{max_lat},{max_lon});
  relation["amenity"]({min_lat},{min_lon},{max_lat},{max_lon});
);
out center;  // 对于 way/relation，取中心点
"""

# 4. 执行查询
result = api.query(query)

# 5. 解析结果成 GeoDataFrame
poi_list = []
for node in result.nodes:
    poi_list.append({
        "id": node.id,
        "lon": float(node.lon),
        "lat": float(node.lat),
        "tags": node.tags
    })
for way in result.ways:
    # way.center_lat / center_lon 来自 out center
    poi_list.append({
        "id": way.id,
        "lon": float(way.center_lon),
        "lat": float(way.center_lat),
        "tags": way.tags
    })
for rel in result.relations:
    poi_list.append({
        "id": rel.id,
        "lon": float(rel.center_lon),
        "lat": float(rel.center_lat),
        "tags": rel.tags
    })

gdf = gpd.GeoDataFrame(
    poi_list,
    geometry=gpd.points_from_xy([p["lon"] for p in poi_list],
                                [p["lat"] for p in poi_list]),
    crs="EPSG:4326"
)

# 6. 可选：筛选类别、保存为 Shapefile / GeoJSON / CSV
gdf.to_file("shanghai_poi_osm.geojson", driver="GeoJSON")
print("POI 总数：", len(gdf))


POI 总数： 21811
